In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Define the model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 3)  # Assuming 3 classes: bored, attentive, confused

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Dummy training loop (Replace with real data)
for epoch in range(1):  # Run real training loop instead
    optimizer.zero_grad()
    dummy_input = torch.randn(1, 3, 224, 224)
    output = model(dummy_input)
    loss = loss_fn(output, torch.tensor([1]))  # Example target class
    loss.backward()
    optimizer.step()

# Save trained model
torch.save(model.state_dict(), "../models/model.pth")
print("✅ Model saved successfully at models/model.pth")


c:\Users\abhis\Downloads\Documents\Learner Engagement Project\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abhis\Downloads\Documents\Learner Engagement Project\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Model saved successfully at models/model.pth


### Define the DAiSEEDataset Class

This cell defines the full custom dataset class for DAiSEE. This definition is required for loading the saved dataset checkpoints and for training the model. Make sure to run this cell before loading any checkpoints.

Below is the code:


In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
from pathlib import Path
from PIL import Image
import logging
from tqdm import tqdm
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

class DAiSEEDataset(Dataset):
    """
    Custom dataset for DAiSEE that reads a CSV with engagement metrics,
    precomputes frame paths based on the ClipID, and loads images & labels.
    """
    def __init__(self, root, csv_path, transform=None, use_cache=True):
        self.root = Path(root)
        self.transform = transform
        self.use_cache = use_cache
        
        self.data = pd.read_csv(csv_path)
        self.frame_paths = []
        self.labels = []
        
        logger.info(f"Processing {len(self.data)} entries from {csv_path.name}")
        for idx, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Loading dataset"):
            try:
                clip_id = str(row['ClipID']).strip()
                if '.avi' in clip_id:
                    clip_id = clip_id.replace('.avi', '')
                parts = clip_id.split('/')
                
                if len(parts) == 1:
                    split_guess = csv_path.stem.replace("Labels", "").strip()
                    frame_path = self.root / split_guess / parts[0] / "frame_0001.jpg"
                elif len(parts) == 2:
                    frame_path = self.root / parts[0] / parts[1] / "frame_0001.jpg"
                elif len(parts) >= 3:
                    frame_path = self.root / parts[0] / parts[1] / parts[2] / "frame_0001.jpg"
                else:
                    logger.debug(f"Skipping row {idx}: unexpected ClipID format: {clip_id}")
                    continue

                if not frame_path.exists():
                    logger.debug(f"Frame path does not exist: {frame_path}")
                else:
                    logger.debug(f"Found frame: {frame_path}")
                
                if frame_path.exists():
                    self.frame_paths.append(frame_path)
                    self.labels.append([
                        float(row['Boredom']),
                        float(row['Engagement']),
                        float(row['Confusion']),
                        float(str(row['Frustration ']).strip())
                    ])
            except Exception as e:
                logger.debug(f"Skipping row {idx} due to error: {e}")
                continue
        
        logger.info(f"Loaded {len(self.frame_paths)} valid frames")
        if len(self.frame_paths) == 0:
            raise ValueError(f"No valid frames found in {csv_path}")
    
    def __len__(self):
        return len(self.frame_paths)
    
    def __getitem__(self, idx):
        img_path = self.frame_paths[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        # Use torchvision.io.read_image for fast loading and convert if needed.
        img_tensor = read_image(str(img_path))
        if self.transform:
            img = self.transform(to_pil_image(img_tensor))
        else:
            img = img_tensor
        return img, label

# Register the dataset class for safe deserialization.
torch.serialization.add_safe_globals(["DAiSEEDataset"])


### Load Saved Datasets

This cell loads the full datasets that were saved in 001_data.ipynb (e.g., `train_set.pth`, `val_set.pth`, and `test_set.pth`). Make sure to run this cell before training the model.


In [2]:
import torch
import os
from torch.utils.data import DataLoader

# Define the path where the dataset checkpoints were saved.
FRAMES_ROOT = "C:/Users/abhis/Downloads/Documents/Learner Engagement Project/data/DAiSEE/ExtractedFrames"

# Load the saved datasets using the full DAiSEEDataset class.
train_dataset = torch.load(os.path.join(FRAMES_ROOT, "train_set.pth"), weights_only=False)
val_dataset = torch.load(os.path.join(FRAMES_ROOT, "val_set.pth"), weights_only=False)
test_dataset = torch.load(os.path.join(FRAMES_ROOT, "test_set.pth"), weights_only=False)

print("✅ Datasets loaded successfully!")


✅ Datasets loaded successfully!


### Create DataLoaders

This cell creates DataLoaders for the training, validation, and testing datasets. These loaders will feed batches of data to your ML/DL models.


In [3]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
NUM_WORKERS = 4  # On Windows, consider using 0 if you encounter issues

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("✅ DataLoaders created successfully!")

✅ DataLoaders created successfully!
